In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import tensorflow as tf
for device in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(device, True)

In [4]:
import omegaconf
import numpy as np
import matplotlib.pyplot as plt
from ganime.data.experimental import ImageDataset, VideoDataset
from ganime.visualization.videos import display_videos
from ganime.visualization.images import display_images
from ganime.model.vqgan_clean.experimental.net2net import Net2Net
import tensorflow_addons as tfa
from datetime import datetime
from tqdm import tqdm
from pyprojroot.pyprojroot import here

#tf.get_logger().setLevel('ERROR')

2022-06-06 16:00:34.025679: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-06 16:00:34.400237: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14252 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [6]:
cfg = omegaconf.OmegaConf.load(here("configs/moving_mnist_image_transformer_huggingface.yaml"))
#cfg = omegaconf.OmegaConf.load(here("configs/default_transformer.yaml"))
batch_size = 8

In [7]:
dataset_length = 10000
num_batch = dataset_length / batch_size

In [8]:
def preprocess(element):
    element = tf.reshape(element, (tf.shape(element)[0], tf.shape(element)[1], tf.shape(element)[2], 3))
    element = tf.cast(element, tf.float32) / 255.0
    first_frame = element[0:1,...]
    last_frame = element[-1:,...]
    
    y = element[1:,...]
    
    first_last_frame = tf.concat([first_frame, last_frame], axis=0)
    
    return {"video": element, "n_frames": tf.shape(element)[0]}

In [9]:
dataset = VideoDataset("../../../data/moving_mnist_tfrecords").load()
dataset = dataset.shuffle(dataset_length, reshuffle_each_iteration=True).map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [10]:
train_size = int(num_batch * 0.8)
validation_size = int(num_batch * 0.1)
test_size = int(num_batch * 0.1)

In [11]:
train_ds = dataset.take(train_size)
validation_ds = dataset.skip(train_size).take(validation_size)
test_ds = dataset.skip(train_size + validation_size).take(validation_size)

In [12]:
train_sample_data = next(train_ds.as_numpy_iterator())
validation_sample_data = next(validation_ds.as_numpy_iterator())

In [13]:
train_sample_data["n_frames"]

array([20, 20, 20, 20, 20, 20, 20, 20], dtype=int32)

In [14]:
from ganime.utils.callbacks import TensorboardVideo, get_logdir
import os

logdir = get_logdir("../../../logs/ganime/", experiment_name="transformer_mnist_video")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
tensorboard_video_callback = TensorboardVideo(logdir, train_sample_data, validation_sample_data)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=50,
    restore_best_weights=True,
)
checkpointing = tf.keras.callbacks.ModelCheckpoint(os.path.join(logdir, "checkpoint", "checkpoint"), monitor='val_loss', save_best_only=True, save_weights_only=True)
callbacks = [tensorboard_callback, early_stopping, checkpointing, tensorboard_video_callback]

In [15]:
images = train_sample_data["video"][:,0,...]

In [16]:
#with strategy.scope():
model = Net2Net(**cfg["model"])
model.first_stage_model.build(train_sample_data["video"].shape[1:])

Working with z of shape (1, 128, 16, 16) = 32768 dimensions.


2022-06-06 16:00:39.890880: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-06-06 16:00:41.056660: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


VQLPIPSWithDiscriminator running with hinge loss.


2022-06-06 16:00:41.959608: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [17]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [18]:
print_gpu_utilization()

GPU memory occupied: 5513 MB.


In [19]:
model.fit(train_ds, steps_per_epoch=50, epochs=2)

Epoch 1/2


/home/abdalla/GANime/venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("while/PartitionedCall:4", shape=(None,), dtype=int32), values=Tensor("while/PartitionedCall:3", shape=(None, 768), dtype=float32), dense_shape=Tensor("while/PartitionedCall:5", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/abdalla/GANime/venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("while/PartitionedCall:1", shape=(None,), dtype=int64), values=Tensor("while/PartitionedCall:0", shape=(None, 768), dtype=float32), dense_shape=Tensor("while/PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


50/50 [==============================] - 123s 2s/step - loss: 446.0566
Epoch 2/2
 4/50 [=>............................] - ETA: 1:41 - loss: 139.9361

KeyboardInterrupt: 

In [ ]:
#model.train_step(train_sample_data)

In [20]:
print_gpu_utilization()

GPU memory occupied: 9609 MB.


In [ ]:
generated_logits = model.process_video(train_sample_data["video"], train_sample_data["n_frames"])

In [ ]:
generated_video = [model.get_image(logits, shape=(32, 8, 8, 128)) for logits in generated_logits]

In [ ]:
display_videos(tf.stack(generated_video, axis=1))

In [ ]:
target_quant, target_indices = model.encode_to_z(train_sample_data["video"][:, 0,...])

In [ ]:
target_indices

In [ ]:
generated_logits